In [70]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from datasets import load_dataset
import torchvision
from torchvision import transforms
import numpy as np


In [71]:
### Hyperparams 
batch_size = 64
num_workers = 4
validation_ratio = 0.2

In [72]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])


In [73]:
path = r'C:\python\datasets_storage'
test_data = torchvision.datasets.CIFAR100(root=path,train=False,download=True, transform=transform)
dataset_train = torchvision.datasets.CIFAR100(root=path,train=True,download=True, transform=transform)

train_data,validation_data=torch.utils.data.random_split(dataset_train,[int((1-validation_ratio)*len(dataset_train)), int((validation_ratio)*len(dataset_train))])
print(len(validation_data))
print(len(train_data))

Files already downloaded and verified
Files already downloaded and verified
10000
40000


In [74]:
train_loader = DataLoader(train_data, batch_size=batch_size, pin_memory=True, shuffle=True)
val_loader = DataLoader(validation_data, batch_size=batch_size, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size,shuffle=False, pin_memory=True)

In [75]:
def check_device():
    if torch.cuda.is_available:
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def move_device(tensor,device):
    if isinstance(tensor,(list,tuple)):    
      return[move_device(element,device) for element in tensor]  
    return tensor.to(device,non_blocking=True)
class DeviceDataloader():
    def __init__(self,dataloader,device):
        self.dataloader = dataloader
        self.device = device
    def __iter__(self):
        # transfer ecach batch and return
        for i in self.dataloader:
           yield move_device(i, self.device)
    def __len__(self):

        return len(self.dataloader)
device = check_device()

train_dl = DeviceDataloader(train_loader, device)
test_dl = DeviceDataloader(test_loader, device)
val_dl  = DeviceDataloader(val_loader, device)
#len(train_dl)

In [76]:
class CNN_model(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(CNN_model,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(32), # out channels, helps normalize batches and imporves overall better performance
            nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2),
            nn.Dropout(dropout_rate)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2),
            nn.Dropout(dropout_rate)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Dropout(dropout_rate)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Dropout(dropout_rate)
        )
        self.hidden_layer = nn.Linear(256 * 2 * 2, 512)
        self.output_layer = nn.Linear(512, 100)
    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2(output)
        output = self.conv3(output)
        output = self.conv4(output)
        #flattening 
        output = output.view(-1, 256 * 2 * 2)
        # fully connected layers
        output = self.hidden_layer(output)
        output = self.output_layer(output)
        return output
model = CNN_model()
print(model)

CNN_model(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  

In [77]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr= 3e-3, momentum=0.9)

In [78]:
print(torch.cuda.is_available())
print(torch.version.cuda if torch.cuda.is_available() else "CUDA not found")

True
12.1


In [79]:
print('Starting...')
epochs = 2
model = model.to(device)
def model_train(model, train_loader, criterion, optimizer):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for batch_idx,(images,targets) in enumerate(train_loader):
            optimizer.zero_grad()
            images,targets = images.to(device), targets.to(device)
            outputs = model(images)
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() ### compares predicted lables to true lables and returns True/False
            batch_idx += 1
        
            if batch_idx % 100 == 0:
                print(f'Epoch" {epoch + 1}/{epochs} | Loss:{running_loss} | '
                      f'Loss: {running_loss / (batch_idx+1):.4f} | '
                      f'Acc_train: {100.*correct/total:.2f}% | '
                      f'Batch: {batch_idx / len(train_loader)}')
def model_val(model, val_loader):
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        for images, targets in val_loader:
            with torch.no_grad():
                images,targets = images.to(device), targets.to(device)
                outputs = model(images)
    
                loss = criterion(outputs, targets)
    
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
                return val_loss/len(val_loader), 100.* correct/total
        print('Finished!')

Starting...


In [80]:
model_train(model,train_loader, criterion, optimizer)
val_loss, val_acc = model_val(model, val_loader)

Epoch" 1/2 | Loss:454.0982165336609 | Loss: 4.4960 | Acc_train: 2.55% | Batch: 0.16
Epoch" 1/2 | Loss:872.953848361969 | Loss: 4.3431 | Acc_train: 4.41% | Batch: 0.32
Epoch" 1/2 | Loss:1275.6716899871826 | Loss: 4.2381 | Acc_train: 5.66% | Batch: 0.48
Epoch" 1/2 | Loss:1666.998646736145 | Loss: 4.1571 | Acc_train: 6.83% | Batch: 0.64
Epoch" 1/2 | Loss:2047.979740858078 | Loss: 4.0878 | Acc_train: 7.74% | Batch: 0.8
Epoch" 1/2 | Loss:2421.242255926132 | Loss: 4.0287 | Acc_train: 8.58% | Batch: 0.96
Epoch" 2/2 | Loss:364.2016565799713 | Loss: 3.6060 | Acc_train: 13.88% | Batch: 0.16
Epoch" 2/2 | Loss:722.3185751438141 | Loss: 3.5936 | Acc_train: 14.62% | Batch: 0.32
Epoch" 2/2 | Loss:1073.6670396327972 | Loss: 3.5670 | Acc_train: 15.25% | Batch: 0.48
Epoch" 2/2 | Loss:1417.8010609149933 | Loss: 3.5357 | Acc_train: 15.82% | Batch: 0.64
Epoch" 2/2 | Loss:1761.9589140415192 | Loss: 3.5169 | Acc_train: 16.24% | Batch: 0.8
Epoch" 2/2 | Loss:2099.3325831890106 | Loss: 3.4931 | Acc_train: 16.79

In [81]:
print(f'Val_loss: {val_loss:.4f}, Val_acc: {val_acc}%')

Val_loss: 0.0219, Val_acc: 17.1875%
